In [120]:

from pandas import read_csv
from pandas import datetime
import math  
import sklearn.metrics 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import factorize

<ipython-input-120-09097fd39d5b>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [121]:
df = pd.read_csv('employee_promotion.csv')
df.head(5)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49.0,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60.0,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50.0,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50.0,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73.0,0


In [122]:
df.shape

(54808, 13)

In [123]:
df = df.copy()

### Data Collection

In [124]:
(df['gender'].value_counts())


m    38496
f    16312
Name: gender, dtype: int64

##### The data of females need to be collected because the above result shows the bias in the given dataset. The data can be collected from multtiple websites or companies where the females are promoted. The important thing is that we have to select only those attributes that is in given dataset.

#Data Cleaning

In [125]:
df.dtypes

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
awards_won                int64
avg_training_score      float64
is_promoted               int64
dtype: object

In [126]:
df.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won', 'avg_training_score', 'is_promoted'],
      dtype='object')

#Changing the datatypes of several column in categories.

Checking the unique values in each features and if it less then changing the datatype to caategories. The benefit of making in categories is that the model would easily understand to give result on new datapoint by just analyzing the each category.

In [127]:
df['department'].unique()

array(['Sales & Marketing', 'Operations', 'Technology', 'Analytics',
       'R&D', 'Procurement', 'Finance', 'HR', 'Legal'], dtype=object)

In [128]:
Department_type = pd.CategoricalDtype(categories = ['Sales & Marketing', 'Operations', 'Technology', 'Analytics','R&D', 'Procurement', 'Finance', 'HR', 'Legal'])
df['department'] = df['department'].astype(Department_type)

#During applying the model i was facing eroor that could not convert string to float. Therefore,i have to make it in numerical form.

We do the encoding because machine unable to understand the categorical values. In this part we are going to apply Ordinal categorical encoding in which we will use label encoding method to encode the department column. The label encoder will give unique rank to every category in the department and when the we want to predict the promotion of new datapoint then the model automatically understand and decide the salary of data point according to unique rank in department column.

In [129]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
df['department'] = lab.fit_transform(df['department'])
df.head(5)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score,is_promoted
0,65438,7,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49.0,0
1,65141,4,region_22,Bachelor's,m,other,1,30,5.0,4,0,60.0,0
2,7513,7,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50.0,0
3,2542,7,region_23,Bachelor's,m,other,2,39,1.0,10,0,50.0,0
4,48945,8,region_26,Bachelor's,m,other,1,45,3.0,2,0,73.0,0


In [130]:
df['region'].unique()

array(['region_7', 'region_22', 'region_19', 'region_23', 'region_26',
       'region_2', 'region_20', 'region_34', 'region_1', 'region_4',
       'region_29', 'region_31', 'region_15', 'region_14', 'region_11',
       'region_5', 'region_28', 'region_17', 'region_13', 'region_16',
       'region_25', 'region_10', 'region_27', 'region_30', 'region_12',
       'region_21', 'region_8', 'region_32', 'region_6', 'region_33',
       'region_24', 'region_3', 'region_9', 'region_18'], dtype=object)

we will drop region column because it is not describing the information of anything i.e. region of what is that city, region in company

In [131]:
df['education'].unique()

array(["Master's & above", "Bachelor's", nan, 'Below Secondary'],
      dtype=object)

In [132]:
Education_type = pd.CategoricalDtype(categories = ["Master's & above", "Bachelor's","Below Secondary"])
df['education'] = df['education'].astype(Education_type)

Doing encoding for education because the model is not working

In [133]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
df['education'] = lab.fit_transform(df['education'])

In [134]:
df['gender'].unique()

array(['f', 'm'], dtype=object)

In [135]:
Gender_type = pd.CategoricalDtype(categories = ['f', 'm'])
df['gender'] = df['gender'].astype(Gender_type)

In [136]:
df['recruitment_channel'].unique()

array(['sourcing', 'other', 'referred'], dtype=object)

In [137]:
Recruitment_type = pd.CategoricalDtype(categories = ['sourcing', 'other', 'referred'])
df['recruitment_channel'] = df['recruitment_channel'].astype(Recruitment_type)

In [138]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
df['recruitment_channel'] = lab.fit_transform(df['recruitment_channel'])

In [139]:
df['is_promoted'].unique()

array([0, 1])

In [140]:
Promoted_type = pd.CategoricalDtype(categories = [0,1])
df['is_promoted'] = df['is_promoted'].astype(Promoted_type)

#Checking the datatype now

In [141]:
df.dtypes

employee_id                int64
department                 int64
region                    object
education                  int64
gender                  category
recruitment_channel        int64
no_of_trainings            int64
age                        int64
previous_year_rating     float64
length_of_service          int64
awards_won                 int64
avg_training_score       float64
is_promoted             category
dtype: object

Deleting the number of training column because it is not giving any type of corelation with the target Variable

In [142]:
df.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [143]:
print(df[['no_of_trainings','is_promoted']].corr())

                 no_of_trainings
no_of_trainings              1.0


In [144]:
df = df.drop(columns=['no_of_trainings'])

##### Deleting the avg_traing score column because it also not shows relation 

---

In [145]:
print(df[['avg_training_score','is_promoted']].corr())

                    avg_training_score
avg_training_score                 1.0


In [146]:
df = df.drop(columns=['avg_training_score'])

Deleting the region column

In [147]:
df = df.drop(columns=['region'])

##### Replacing the m with male and f with female in gender column. Because it willmake more sense and easily understandable.

In [148]:
df['gender'].unique()

['f', 'm']
Categories (2, object): ['f', 'm']

In [149]:
df['gender'].replace('f','Female',inplace = True) 
df['gender'].replace('m','Male',inplace = True) 

In [150]:
df.head(5)

,employee_id,department,education,gender,recruitment_channel,age,previous_year_rating,length_of_service,awards_won,is_promoted
0,65438,7,2,Female,2,35,5.0,8,0,0
1,65141,4,0,Male,0,30,5.0,4,0,0
2,7513,7,0,Male,2,34,3.0,7,0,0
3,2542,7,0,Male,0,39,1.0,10,0,0
4,48945,8,0,Male,0,45,3.0,2,0,0


#Feature Engineering

Feature encoding of gender column

In [151]:
df = pd.get_dummies(df, columns=["gender"], prefix='gender')

#### Checking null values

In [152]:
df.isnull().sum() 

employee_id                0
department                 0
education                  0
recruitment_channel        0
age                        0
previous_year_rating    4124
length_of_service          0
awards_won                 0
is_promoted                0
gender_Female              0
gender_Male                0
dtype: int64

## Handling missing values

In [153]:
df.columns

Index(['employee_id', 'department', 'education', 'recruitment_channel', 'age',
       'previous_year_rating', 'length_of_service', 'awards_won',
       'is_promoted', 'gender_Female', 'gender_Male'],
      dtype='object')

I'm going to fill the value of previous _year_rating column with the mean of column

In [154]:
df['previous_year_rating'] = df['previous_year_rating'].fillna(df['previous_year_rating'].mean())

About the education column, it is in categorical form so i can't fill missing values with mean. Now i'm going to apply mode.

In [155]:
df['education'] = df['education'].fillna(df['education'].mode()[0])

#Now checking the missing values

In [156]:
df.isnull().sum()

employee_id             0
department              0
education               0
recruitment_channel     0
age                     0
previous_year_rating    0
length_of_service       0
awards_won              0
is_promoted             0
gender_Female           0
gender_Male             0
dtype: int64

In [157]:
df_ready = df

In [158]:
df_ready.dtypes

employee_id                int64
department                 int64
education                  int64
recruitment_channel        int64
age                        int64
previous_year_rating     float64
length_of_service          int64
awards_won                 int64
is_promoted             category
gender_Female              uint8
gender_Male                uint8
dtype: object

#Making the columns in series so that the splitting can done properly.

In [159]:
df_ready.columns

Index(['employee_id', 'department', 'education', 'recruitment_channel', 'age',
       'previous_year_rating', 'length_of_service', 'awards_won',
       'is_promoted', 'gender_Female', 'gender_Male'],
      dtype='object')

In [160]:
df_ready = df_ready[['employee_id', 'department','education','recruitment_channel', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won','gender_Female',
       'gender_Male','is_promoted']]

In [161]:
df_ready.head(5)

,employee_id,department,education,recruitment_channel,age,previous_year_rating,length_of_service,awards_won,gender_Female,gender_Male,is_promoted
0,65438,7,2,2,35,5.0,8,0,1,0,0
1,65141,4,0,0,30,5.0,4,0,0,1,0
2,7513,7,0,2,34,3.0,7,0,0,1,0
3,2542,7,0,0,39,1.0,10,0,0,1,0
4,48945,8,0,0,45,3.0,2,0,0,1,0


#Modelling

#Splitting of data into target and features.

In [162]:
y = df_ready['is_promoted']
X = df_ready.iloc[:, 0:10]   # It will add features until 12 index in the X variable.
X.head(2)

,employee_id,department,education,recruitment_channel,age,previous_year_rating,length_of_service,awards_won,gender_Female,gender_Male
0,65438,7,2,2,35,5.0,8,0,1,0
1,65141,4,0,0,30,5.0,4,0,0,1


Splitting into traing and testing

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, test_size = .30)

#Instatiate Decision Tree Classifier

In [164]:
from sklearn.tree import DecisionTreeClassifier
DecisionTree_model = DecisionTreeClassifier()

#Fitting

In [165]:
DecisionTree_model.fit(X_train,y_train)

DecisionTreeClassifier()

In [166]:
y_predict = DecisionTree_model.predict(X_test)

In [167]:
accuracy_score(y_test, y_predict)

0.8471690080885483

odel is giving better accuracy and it doesn't showing overfitting.

##Random Forest Classifier

In [168]:
from sklearn.ensemble import RandomForestClassifier

In [169]:
RF_cls = RandomForestClassifier()

In [170]:
RF_cls.fit(X_train, y_train)

RandomForestClassifier()

In [171]:
y_prediction = RF_cls.predict(X_test)

In [172]:
accuracy_score(y_test, y_prediction)

0.9106002554278416

##Random Forest is also performs well.